In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

### Defining custom transforms

In [20]:
class WineDataset(Dataset):
    
    def __init__(self, transform=None):
        #Data loading
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = xy[:,1:]
        self.y = xy[:,[0]]
        self.n_samples = xy.shape[0]
        self.transform = transform
        
    def __getitem__(self, index):
        #Return a particular element of the given index
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        #Number of samples
        return self.n_samples
    

class ToTensor:
    def __call__(self, sample):
        features, labels = sample
        return torch.from_numpy(features), torch.from_numpy(labels)
    
class MulTransform:
    def __init__(self,f):
        self.factor = f
        
    def __call__(self, sample):
        features, labels = sample
        
        features *= self.factor
        
        return features, labels


composite = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
    
dataset = WineDataset(transform=composite)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])


In the previous exercise, we would've converted the loaded numpy array to tensor in the `WineDataset's` `init` function. But here, we pass another argument called `transform` to the `init` function of the `WineDataset`, which is a callable instance of any transform class that we define. We have defined a class `ToTensor`, that converts a sample to a tensor, and have used this in the `WineDataset` class's `getitem` function to return a sample after converting it to a tensor. Similarly we have implemented a transformer class that transforms the data by multiplying by a factor `f`.

We can create composite transforms with the help of torchvision's `Compose` module, which accepts a list of transforms as its argument and applies the transforms in the order given in the list. There is no need to create an instance of composite as it by itself is an object that is callable. So we simply pass it as the `transform` argument.